In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

tf.keras.backend.set_floatx('float32')

Load, scale and shuffle dataset

In [2]:
train_df = pd.read_csv('https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv')
test_df = pd.read_csv('https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv')

scale_factor = 1000
train_df['median_house_value'] /= scale_factor
test_df['median_house_value'] /= scale_factor

train_df = train_df.reindex(np.random.permutation(train_df.index))

Latitude and longitude input layers

In [3]:
inputs = {
    'latitude' : tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='latitude'),
    'longitude': tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name='longitude')
}

Model creation, training and plotting functions

In [7]:
def create_model(inputs, outputs, learning_rate):
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=learning_rate), loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model


def train_model(model, dataset, epochs, batch_size, label_name):
    features = {name:np.array(value) for name, value in dataset.items()}
    label = np.array(features.pop(label_name))

    history = model.fit(x=features, y=label, batch_size=batch_size, epochs=epochs, shuffle=True)
    epochs = history.epoch
    rmse = pd.DataFrame(history.history)['root_mean_squared_error']

    return epochs, rmse


def plot_loss(epochs, rmse):
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.94, rmse.max()* 1.05])
  plt.show()

Train model

In [8]:
learning_rate = 0.05
epochs = 30
batch_size = 100
label_name = 'median_house_value'

preprocessing_layer = tf.keras.layers.Concatenate()(inputs.values())
dense_output = tf.keras.layers.Dense(units=1, name='dense_layer')(preprocessing_layer)
outputs = {'dense_output': dense_output}

my_model = create_model(inputs, outputs, learning_rate)
tf.keras.utils.plot_model(my_model, 'model.png', show_shapes=True)
epochs, rmse = train_model(my_model, train_df, epochs, batch_size, label_name)

my_model.summary(expand_nested=True)
plot_loss(epochs, rmse)

AttributeError: module 'keras.optimizers' has no attribute 'experimental'